# COD10K Dataset

In [1]:
import os
import numpy as np
import cv2
from glob import glob
# from tqdm import tqdm
import csv
from torchvision.io import read_image
import torch
from torchvision.ops import masks_to_boxes
from PIL import Image


In [2]:
JUST_COD10K = True
# Convert Pascal_Voc bb to Yolo

def pascal_voc_to_yolo(new_bbox, image_w = 416, image_h = 416):
  # Bbox format (xmin, ymin, xmax,ymax)  
  x1,y1 = new_bbox[0],new_bbox[1]
  x2,y2 = new_bbox[2],new_bbox[3]
  label = new_bbox[4]
  w = x2 - x1
  h = y2 - y1
  b_center_x = (x1+x2) / (2*image_w) 
  b_center_y = (y1+y2) / (2*image_h)
  b_width    = w/image_w
  b_height   = h/image_h
      
  return ("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(label,b_center_x, b_center_y, b_width, b_height))


In [3]:
# Import Test dataset
!gdown -- 1QEGnP9O7HbN_2tH999O3HRIsErIVYalx

Downloading...
From: https://drive.google.com/uc?id=1QEGnP9O7HbN_2tH999O3HRIsErIVYalx
To: /content/TestDataset.zip
100% 700M/700M [00:06<00:00, 101MB/s]


In [4]:
!unzip TestDataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-597.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-591.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-590.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-588.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-585.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-583.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-582.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-580.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-579.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-577.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-576.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish

In [5]:
!gdown -- 1D9bf1KeeCJsxxri6d2qAC7z6O1X_fxpt

Downloading...
From: https://drive.google.com/uc?id=1D9bf1KeeCJsxxri6d2qAC7z6O1X_fxpt
To: /content/TrainDataset.zip
100% 1.10G/1.10G [00:11<00:00, 95.8MB/s]


In [6]:
!unzip TrainDataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: TrainDataset/GT/camourflage_00199.png  
  inflating: TrainDataset/GT/camourflage_00198.png  
  inflating: TrainDataset/GT/camourflage_00195.png  
  inflating: TrainDataset/GT/camourflage_00194.png  
  inflating: TrainDataset/GT/camourflage_00193.png  
  inflating: TrainDataset/GT/camourflage_00192.png  
  inflating: TrainDataset/GT/camourflage_00191.png  
  inflating: TrainDataset/GT/camourflage_00190.png  
  inflating: TrainDataset/GT/camourflage_00189.png  
  inflating: TrainDataset/GT/camourflage_00187.png  
  inflating: TrainDataset/GT/camourflage_00185.png  
  inflating: TrainDataset/GT/camourflage_00184.png  
  inflating: TrainDataset/GT/camourflage_00182.png  
  inflating: TrainDataset/GT/camourflage_00181.png  
  inflating: TrainDataset/GT/camourflage_00180.png  
  inflating: TrainDataset/GT/camourflage_00179.png  
  inflating: TrainDataset/GT/camourflage_00178.png  
  inflating: TrainDataset/GT/camourflage_00177.p

In [7]:
path = "TestDataset/COD10K/Imgs/"
# Put all the test images into a dictionary based on their classes
files = {}
for img in os.listdir(path):
  class_name = img.rstrip(".jpg").split("-")[5]
  if class_name not in files:
    files[class_name] = set()
  files[class_name].add(os.path.join(path,img))
print(files)

{'GhostPipefish': {'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-409.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-327.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-334.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-391.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-403.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-370.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-395.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-369.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-346.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-315.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-347.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-385.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-9-GhostPipefish-379.jpg', 'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic

In [8]:
path = "TrainDataset/Imgs/"
for img in os.listdir(path):
  # Only gather images from COD10K-CAM dataset that are annotated
  if "COD10K-CAM" in img:
    class_name = img.rstrip(".jpg").split("-")[5]
    if class_name not in files:
      files[class_name] = set()
    files[class_name].add(os.path.join(path,img))

In [9]:
from sklearn.model_selection import train_test_split
def camo_train_test_split(X):
  y = [0]*len(X)
  X_train, X_test, _, _ = train_test_split( X, y, test_size=0.2, random_state=42)
  return X_train,X_test
files_refined = {"train":{},"test":{}}
camo_train_test_split(list(files["Slug"]))

(['TrainDataset/Imgs/COD10K-CAM-1-Aquatic-17-Slug-1148.jpg',
  'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-17-Slug-1151.jpg',
  'TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-17-Slug-1149.jpg',
  'TrainDataset/Imgs/COD10K-CAM-1-Aquatic-17-Slug-1152.jpg'],
 ['TrainDataset/Imgs/COD10K-CAM-1-Aquatic-17-Slug-1150.jpg'])

In [10]:
# Split all files into training and test dataset
for class_name in files:
  train,test =camo_train_test_split(list(files[class_name]))
  files_refined["train"][class_name] = train
  files_refined["test"][class_name] = test

In [11]:
!rm -rf Dataset

In [12]:
# Create neccessary folders
os.mkdir("Dataset")
os.mkdir(os.path.join("Dataset","Train"))
os.mkdir(os.path.join("Dataset","Train","Images"))
os.mkdir(os.path.join("Dataset","Train","Labels"))
os.mkdir(os.path.join("Dataset","Test"))
os.mkdir(os.path.join("Dataset","Test","Images"))
os.mkdir(os.path.join("Dataset","Test","Labels"))

In [13]:
# Get all the labels
labels = sorted(files_refined["test"].keys())

In [14]:
labels

['Ant',
 'Bat',
 'BatFish',
 'Bee',
 'Beetle',
 'Bird',
 'Bittern',
 'Bug',
 'Butterfly',
 'Cat',
 'Caterpillar',
 'Centipede',
 'Chameleon',
 'Cheetah',
 'Cicada',
 'ClownFish',
 'Crab',
 'Crocodile',
 'CrocodileFish',
 'Deer',
 'Dog',
 'Dragonfly',
 'Duck',
 'Fish',
 'Flounder',
 'Frog',
 'FrogFish',
 'Frogmouth',
 'Gecko',
 'GhostPipefish',
 'Giraffe',
 'Grasshopper',
 'Grouse',
 'Heron',
 'Human',
 'Kangaroo',
 'Katydid',
 'LeafySeaDragon',
 'Leopard',
 'Lion',
 'Lizard',
 'Mantis',
 'Mockingbird',
 'Monkey',
 'Moth',
 'Octopus',
 'Other',
 'Owl',
 'Owlfly',
 'Pagurian',
 'Pipefish',
 'Rabbit',
 'Reccoon',
 'Sciuridae',
 'ScorpionFish',
 'SeaHorse',
 'Sheep',
 'Shrimp',
 'Slug',
 'Snake',
 'Spider',
 'StarFish',
 'StickInsect',
 'Stingaree',
 'Tiger',
 'Toad',
 'Turtle',
 'Wolf',
 'Worm']

In [15]:
if not JUST_COD10K:
  labels_MOCA = ['Flower crab spider', 'Polar bear', 'Grasshopper', 'Flounder', 'Snowy owl', 'Goat', 'Black cat', 'Bear', 'Spider tailed horned viper', 'Rodent x', 'Potoo', 'Arctic fox', 'Devil scorpionfish', 'Arabian horn viper', 'Snow leopard', 'Peacock flounder', 'Stick insect', 'Seal', 'Snowshoe hare', 'Leaf katydid', 'Scorpionfish', 'Jerboa', 'Cuttlefish', 'Orchid mantis', 'Lion cub', 'Rabbit', 'Rusty spotted cat', 'Eastern screech owl', 'Flatfish', 'Hedgehog', 'Groundhog', 'Starfish', 'Pygmy seahorse', 'Plaice', 'Arctic wolf', 'Copperhead snake', 'Ibex', 'Polar bear cub', 'White tailed ptarmigan', 'Lioness', 'Dead leaf butterfly', 'Fossa', 'Smallfish', 'Turtle', 'Chameleon', 'Marine iguana', 'Crab', 'Nile monitor', 'Moth', 'Horned owl', 'Mongoose', 'Hermit crab', 'Elephant', 'Sand cat', 'Wolf', 'Octopus', 'Desert fox', 'Mountain goat', 'Leaf tail gecko', 'Meerkat', 'Sole', 'Lichen katydid', 'Pallas cat', 'Shrimp', 'Markhor', 'Egyptian nightjar', 'Hyena']
  for label_im in labels:
    if label_im not in labels_MOCA:
      labels_MOCA.append(label_im)

  labels = labels_MOCA

In [16]:
len(labels)

69

In [17]:
!pip install "opencv-python-headless<4.3"
!pip install -U git+https://github.com/albumentations-team/albumentations

     |████████████████████████████████| 21.6 MB 107.2 MB/s 


  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-d9dybjnu
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-d9dybjnu
  Created wheel for albumentations: filename=albumentations-1.1.0-py3-none-any.whl size=112723 sha256=09784d132dd840705e3155dcb0f68a65142ec5d2f14348dff4a2022d16058b20
  Stored in directory: /tmp/pip-ephem-wheel-cache-dth64xov/wheels/3a/25/ed/ec3b518e7a332d7f0a3bb37c280e1b784cf2f79b94b3c7d00b
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [18]:

import albumentations
def resize_image(img_arr, bboxes, h, w):
  """
  :param img_arr: original image as a numpy array
  :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
  :param h: resized height dimension of image
  :param w: resized weight dimension of image
  :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
  """
  # create resize transform pipeline
  transform = albumentations.Compose(
      [albumentations.Resize(height=h, width=w, always_apply=True)],
      bbox_params=albumentations.BboxParams(format='pascal_voc'))

  transformed = transform(image=img_arr, bboxes=bboxes)
  return transformed

In [19]:
# Offsetted so doesn't clash with MoCA images
img_num = 7617

import shutil
for subset_type, folder_name in [("train","Train"),("test","Test")]:
  for class_name in sorted(files_refined[subset_type]):
    for file_name in sorted(files_refined[subset_type][class_name]):
      
      file_path = file_name.replace("Imgs","GT").replace(".jpg",".png")
      img = read_image(file_path)
      bboxes = np.array(masks_to_boxes(img))
      class_label = labels.index(class_name)
      new_bboxes = []
      # Just get 1 bbox. For some reason 3 bboxes which were identical were obtained
      for arr in bboxes:
        new_bboxes.append(np.array([arr[0],arr[1],arr[2],arr[3],class_label]))
        break
      bboxes = np.array(new_bboxes)
      img = np.array(Image.open(file_name))
      transformed_dict = resize_image(img, bboxes, 416, 416)
      transformed_arr = transformed_dict["image"]
      bboxes = np.array(list(map(list, transformed_dict["bboxes"]))).astype(float)
      file_name_in_dir = file_name.split("/")[-1]
      class_name = file_name.rstrip(".jpg").split("-")[5]
      end_path_img = os.path.join("Dataset",folder_name,"Images",str(img_num)+".jpg")
      end_path_label = os.path.join("Dataset",folder_name,"Labels",str(img_num)+".txt")
      img_num+=1
      img = Image.fromarray(transformed_arr)
      img.save(end_path_img)

      with open(end_path_label,"w") as f:
        for bbox in bboxes:
          f.write(f"{pascal_voc_to_yolo(bbox)}\n")
          break



In [20]:
if JUST_COD10K:
  !zip Dataset_COD10KOnly.zip -r Dataset/
else:
  !zip Dataset_COD10K.zip -r Dataset/


Streaming output truncated to the last 5000 lines.
  adding: Dataset/Train/Images/8431.jpg (deflated 0%)
  adding: Dataset/Train/Images/10951.jpg (deflated 1%)
  adding: Dataset/Train/Images/10480.jpg (deflated 0%)
  adding: Dataset/Train/Images/7759.jpg (deflated 0%)
  adding: Dataset/Train/Images/10526.jpg (deflated 1%)
  adding: Dataset/Train/Images/7940.jpg (deflated 1%)
  adding: Dataset/Train/Images/11334.jpg (deflated 1%)
  adding: Dataset/Train/Images/8475.jpg (deflated 1%)
  adding: Dataset/Train/Images/8678.jpg (deflated 0%)
  adding: Dataset/Train/Images/10530.jpg (deflated 0%)
  adding: Dataset/Train/Images/9643.jpg (deflated 0%)
  adding: Dataset/Train/Images/10033.jpg (deflated 0%)
  adding: Dataset/Train/Images/8640.jpg (deflated 0%)
  adding: Dataset/Train/Images/9640.jpg (deflated 1%)
  adding: Dataset/Train/Images/9908.jpg (deflated 1%)
  adding: Dataset/Train/Images/10734.jpg (deflated 15%)
  adding: Dataset/Train/Images/10990.jpg (deflated 0%)
  adding: Dataset/Trai

In [21]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [22]:
if JUST_COD10K:
  !cp Dataset_COD10KOnly.zip "drive/My Drive/"
else:
  !cp Dataset_COD10K.zip "drive/My Drive/"